In [293]:
from copy import deepcopy
import json
import os
from typing import List
import uuid
import re
import pandas as pd

## Utilidades

In [294]:
def drop_duplicates(ls: list) -> list:
    return list(filter(None, set(ls)))

def flatten(ls: list) -> list:
    return [item for sublist in ls for item in sublist]

def remove(ls: list, to_remove: list) -> list:
    ls_copy = deepcopy(ls)
    for el in to_remove:
        if el in ls_copy:
            ls_copy.remove(el)
    return ls_copy

## I/O

In [295]:
def load_perguntas():
    df = pd.read_excel("results/Perguntas.xlsx", sheet_name="finais")
    df = df.dropna(subset=["Resposta"])
    subs = {
        "Pergunta": "pergunta",
        "Resposta": "resposta",
        "Intenção": "intent",
        "Rótulos": "rótulos",
        "Modificador": "modificador",
        "Substantivo": "substantivo",
        "Recipiente": "recipiente",
        "Elocuções": "examples",
    }
    cols = list(subs.keys())
    df = df[cols]
    df = df.rename(columns=subs)
    df = df.fillna("")
    return df

def load_skill(skill_path):
    with open(skill_path, "r", encoding="utf-8") as f:
        skill = json.load(f)
    return skill

def save_skill(skill_path, skill):
    skill_root, skill_ext = os.path.splitext(skill_path)
    new_skill_path = f"{skill_root}2{skill_ext}"
    with open(new_skill_path, "w", encoding="utf-8") as f:
        json.dump(skill, f, ensure_ascii=False)

## Intenções

In [296]:
def get_intents(df: pd.DataFrame):
    subset = ["intent", "pergunta", "examples"]
    records = df[subset].to_dict(orient="records")
    intents = [
        {
            "intent": record["intent"],
            "examples": get_examples(record),
            "description": "",
        }
        for record in records
    ]
    return intents

def get_examples(record: dict) -> List:
    # a própria pergunta é um exemplo
    out = [{"text": record["pergunta"]}]
    # tudo que está em Elocuções é exemplo também
    if record["examples"]:
        out += [{"text": exemplo} for exemplo in record["examples"].split("--")]
    return out

## Entidades

In [297]:
def get_entity_values(series):
    records = series.drop_duplicates().to_list()
    records = [r.split("-") for r in records]
    records = flatten(records)
    records = drop_duplicates(records)
    values = [
        {"type": "synonyms", "value": record, "synonyms": []} for record in records
    ]
    return values

def get_entities(df):
    subset = ["rótulos", "modificador", "substantivo", "recipiente"]
    entities = [
        {"entity": col, "values": get_entity_values(df[col]), "fuzzy_match": True}
        for col in subset
    ]
    return entities

def mix_entities(a, b):
    """
    Pega sinônimos que foram adicionados a entidades através da interface do Watson
    e copia para as entidades geradas automaticamente da planilha.
    """
    ents_old = deepcopy(a)
    ents_new = deepcopy(b)
    for col in ["modificador", "rótulos", "substantivo", "recipiente"]:
        try:
            ent_old = next(ent for ent in ents_old if ent["entity"] == col)
            ent_new = next(ent for ent in ents_new if ent["entity"] == col)
        except StopIteration:
            continue
        for v_old in ent_old["values"]:
            for v_new in ent_new["values"]:
                if v_new["value"] == v_old["value"]:
                    v_new["synonyms"] = v_old["synonyms"]
    return ents_new

## Nós de diálogo

In [298]:
def get_contextos(rotulos):
    # se algum dos rótulos abaixo estiver presente em qualquer rótulo,
    # ele não é um contexto
    rotulos_nao_contextuais = [
        "fauna",
        "flora",
        "extra",
        "física",
        "símbolo",
        "turismo",
        "engenharia",
        "saúde",
        "geologia",
    ]
    contextos = [contexto for contexto in rotulos if all([rot not in contexto for rot in rotulos_nao_contextuais])]
    return contextos

def plural(palavra: str) -> str:
    return palavra + "s" if palavra[-1] != "s" else palavra

def get_titulo(js: dict) -> str:
    modificador = js["modificador"]
    substantivo = js["substantivo"].replace("-", " ")
    recipiente = js["recipiente"]
    
    contextos = get_contextos(js["rótulos"].split("-"))
    contextos = list(filter(lambda x: x.count("-") == 0, contextos))
    titulo = f"{'/'.join(contextos)}: " if contextos else ""
    titulo += modificador + " "
    
    if modificador in ["efeito"]:
        titulo += f" de {substantivo} em {recipiente}"
    elif modificador in ["maiores", "menores"]:
        substituir = {"produção": "produtores"}
        if recipiente in substituir:
            recipiente = substituir[recipiente]
        substantivo = plural(substantivo)
        titulo += f"{substantivo} {recipiente}"
    elif modificador in ["diferença"]:
        recipiente = recipiente or contextos[0]
        titulo += f"entre {substantivo} e {recipiente}"
    elif modificador in ["existe", "quantidade"]:
        recipiente = recipiente or "no Brasil"
        if substantivo:
            titulo += substantivo + " "
        titulo += f"{recipiente}"
    elif modificador in ["listar"]:
        substituir = {"extinção": "em extinção", "aaz": "na Amazônia Azul"}
        if recipiente in substituir:
            recipiente = substituir[recipiente]
        substantivo = plural(substantivo)
        titulo += f"{substantivo} {recipiente}"
    else:
        if substantivo:
            titulo += substantivo + " "
        if recipiente:
            titulo += recipiente
    titulo = titulo.strip()
    titulo = titulo[0].capitalize() + titulo[1:]
    return titulo

def get_condition(modificador, substantivo, recipiente, contexto=None) -> list:
    cond = f"@modificador:{modificador}"
    if substantivo:
        cond += f" && @substantivo:{substantivo}"
    if recipiente:
        cond += f" && @recipiente:{recipiente}"
    if contexto:
        out = [cond + f' && $contexto=="{contexto}"', cond + f" && @rótulos:{contexto}"]
    else:
        out = [cond]
    return out

def get_condition_string(js):
    modificador = js["modificador"]
    substantivo = js["substantivo"]
    recipiente = js["recipiente"]
    rotulos = js["rótulos"].split("-") + [js["rótulos"]]
    rotulos = drop_duplicates(rotulos)
    contextos = get_contextos(rotulos)

    if contextos:
        conds_adicionais = [get_condition(modificador, substantivo, recipiente, contexto) for contexto in contextos]
    else:
        conds_adicionais = [get_condition(modificador, substantivo, recipiente)]

    conds = [f"#{js['intent']}"] + flatten(conds_adicionais)
    cond_str = " || ".join(conds)
    return cond_str
    

def get_dialog_nodes(df):
    records = df.to_dict(orient="records")
    dialog_nodes = [
        {
            "type": "standard",
            "title": get_titulo(record),
            "output": {
                "generic": [
                    {
                        "values": [{"text": record["resposta"]}],
                        "response_type": "text",
                        "selection_policy": "sequential",
                    }
                ]
            },
            "context": {"contexto": record["rótulos"]},
            "conditions": get_condition_string(record),
            "dialog_node": f"node_{uuid.uuid4().hex[:16]}"
        }
        for record in records
    ]
    
    # atribuir previous siblings
    for i in range(len(dialog_nodes) - 1):
        prev, node = dialog_nodes[i], dialog_nodes[i + 1]
        node["previous_sibling"] = prev["dialog_node"]
        
    return dialog_nodes

def mix_dialog_nodes(old: dict, new: dict) -> dict:
    pass

## Skill

In [299]:
def mix_skills(skill, **kwargs):
    new_skill = deepcopy(skill)
    for k, v in kwargs.items():
        new_skill[k] = v
    return new_skill

## Rodar

In [300]:
def is_flat(x):
    return not (isinstance(x, list) or isinstance(x, dict))

def is_flat_list(x):
    return all(is_flat(y) for y in x)

def mix_list(a, b):
    """
    Realiza um outer join entre duas listas planas ou que contêm dicionários.
    """
    if is_flat_list(a) and is_flat_list(b):
        return drop_duplicates(a + b)
    elif is_flat_list(a) or is_flat_list(b):
        raise ValueError(a, b)
    
    a_dict = to_dict(a)
    b_dict = to_dict(b)
    mixed_dicts = mix_dict(a_dict, b_dict)
    return list(mixed_dicts.values())

def to_dict(ls: List[dict]):
    """
    Transforma uma lista de dicionários em um dicionário de dicionários, 
    inferindo como chave para cada dict algum valor dele que seja presente
    em todos os dicts, único em cada um e plano.
    """
    if len(ls) == 1:
        return {"placeholder_key": ls[0]}
    d = ls[0]
    global_key = None
    for key in d:
        # se a chave for uma das hardcoded
        if key in ["conditions"]:
            global_key = key
            break
        # se o valor dessa chave não for plano
        if not is_flat(d[key]):
            continue
        # se nem todos dicts tiverem essa chave
        if not all(key in d_ for d_ in ls):
            continue
        # se nem todos os dicts tiverem um valor para essa chave
        if not all(bool(d_[key]) for d_ in ls):
            continue
        # se cada dict não tiver um valor único para essa chave
        if not len(set(d_[key] for d_ in ls)) == len(ls):
            continue
        # se já houver uma chave global encontrado
        if global_key:
            raise ValueError(f"Mais de uma possível chave global encontrada para {ls}: {key}, {global_key}")
        global_key = key
        
    if not global_key:
        raise ValueError(f"Não encontrei chave global para {ls}")
    return {d_[global_key]: d_ for d_ in ls}
    
def mix_dict(a, b):
    """
    Retorna um dicionário contendo as chaves e valores de ambos dicionários
    de entrada, dando preferência para os valores de b.
    """
    out = deepcopy(b)
    for k, v in a.items():
        if k not in out:
            out[k] = v
        elif isinstance(v, list):
            if not isinstance(b[k], list):
                raise ValueError(f"a é uma lista em {k}, mas b é {type(b[k])}")
            out[k] = mix_list(v, b[k])
        elif isinstance(v, dict):
            if not isinstance(b[k], dict):
                raise ValueError(f"a é um dict em {k}, mas b é {type(b[k])}")
            out[k] = mix_dict(v, b[k])
        else:
            out[k] = v
    return out
            

In [301]:
df = load_perguntas()
df[~(df["examples"]=="")]

,pergunta,resposta,intent,rótulos,modificador,substantivo,recipiente,examples
56,Por que tartarugas estão em extinção?,"Além dos predadores naturais, a principal amea...",tartaruga--motivo-extinção,tartaruga,motivo,,extinção,Quais são as ameaças às tartarugas?
172,O que é Chordata?,"Chordata, um dos filos do reino Animalia, é ca...",fauna--definir-cordado,fauna,definir,cordado,,O que são cordados?
174,O que é uma cnida?,Cnida (ou nematocisto) é uma cápsula que conté...,fauna--definir-cnida,fauna,definir,cnida,,O que é um nematocisto?


In [302]:
skill_path = "results/skill-Amazônia-Azul.json"
skill = load_skill(skill_path)

In [303]:
new_intents = mix_list(skill["intents"], get_intents(df))
new_intents

[{'intent': 'coral--definir',
  'examples': [{'text': 'O que é um coral?'}],
  'description': ''},
 {'intent': 'coral--composição',
  'examples': [{'text': 'Do que é feito um coral?'}],
  'description': ''},
 {'intent': 'petróleo--maiores-reserva',
  'examples': [{'text': 'Quais são as maiores reservas de petróleo no Brasil?'}],
  'description': ''},
 {'intent': 'gás--maiores-estado-produção',
  'examples': [{'text': 'Que estados produzem mais petróleo?'}],
  'description': ''},
 {'intent': 'petróleo--maiores-estado-produção',
  'examples': [{'text': 'Que estados produzem mais gás?'}],
  'description': ''},
 {'intent': 'petróleo--quantidade',
  'examples': [{'text': 'Quanto petróleo há no Brasil?'}],
  'description': ''},
 {'intent': 'gás--quantidade-consumo',
  'examples': [{'text': 'Qual é o consumo de gás brasileiro?'}],
  'description': ''},
 {'intent': 'petróleo--localização',
  'examples': [{'text': 'Onde o petróleo é encontrado?'}],
  'description': ''},
 {'intent': 'petróleo--q

In [304]:
new_entities = mix_list(skill["entities"], get_entities(df))
new_entities

[{'entity': 'rótulos',
  'values': [{'type': 'synonyms', 'value': 'gás', 'synonyms': []},
   {'type': 'synonyms', 'value': 'fauna', 'synonyms': []},
   {'type': 'synonyms', 'value': 'turismo', 'synonyms': []},
   {'type': 'synonyms', 'value': 'coral', 'synonyms': []},
   {'type': 'synonyms', 'value': 'petróleo', 'synonyms': []},
   {'type': 'synonyms', 'value': 'flora', 'synonyms': []},
   {'type': 'synonyms', 'value': 'extra', 'synonyms': []},
   {'type': 'synonyms', 'value': 'tartaruga', 'synonyms': []}],
  'fuzzy_match': True},
 {'entity': 'modificador',
  'values': [{'type': 'synonyms', 'value': 'localização', 'synonyms': []},
   {'type': 'synonyms', 'value': 'maiores', 'synonyms': []},
   {'type': 'synonyms', 'value': 'quantidade', 'synonyms': []},
   {'type': 'synonyms', 'value': 'existe', 'synonyms': []},
   {'type': 'synonyms', 'value': 'composição', 'synonyms': []},
   {'type': 'synonyms', 'value': 'detalhar', 'synonyms': []},
   {'type': 'synonyms', 'value': 'motivo', 'synony

In [305]:
new_dialog_nodes = mix_list(skill["dialog_nodes"], get_dialog_nodes(df))
new_dialog_nodes

[{'type': 'standard',
  'title': 'Coral: definir',
  'output': {'generic': [{'values': [{'text': 'Corais são cnidários que possuem um exoesqueleto calcário ou de matéria orgânica, o que os diferencia de anêmonas.'}],
     'response_type': 'text',
     'selection_policy': 'sequential'}]},
  'context': {'contexto': 'coral'},
  'conditions': '#coral--definir || @modificador:definir && $contexto=="coral" || @modificador:definir && @rótulos:coral',
  'dialog_node': 'node_e94eb4837dc34cc2',
  'previous_sibling': 'node_4_1633059502031'},
 {'type': 'standard',
  'title': 'Coral: composição',
  'output': {'generic': [{'values': [{'text': 'Um coral pode ser constituído de um ou mais pólipos adultos. É comum que possuam um exoesqueleto calcário, ou seja, sejam bem duros por fora.'}],
     'response_type': 'text',
     'selection_policy': 'sequential'}]},
  'context': {'contexto': 'coral'},
  'conditions': '#coral--composição || @modificador:composição && $contexto=="coral" || @modificador:composi

In [306]:
new_skill = mix_skills(skill, intents=new_intents, entities=new_entities, dialog_nodes=new_dialog_nodes)
save_skill(skill_path, new_skill)